In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
os.environ['KULOKO_DIR'] = '/Users/macico/Dropbox/btc/kuloko'
os.environ['COMMON_DIR'] = os.path.join(os.environ['KULOKO_DIR'], "geco_commons")
os.environ['MONGO_DIR'] = os.path.join(os.environ['COMMON_DIR'] ,"mongodb")
os.environ['LOGDIR'] = os.path.join(os.environ['KULOKO_DIR'], "log")

In [26]:
from pymongo import MongoClient
import configparser
import logging
import logging.config

KULOKO_DIR=os.environ['KULOKO_DIR'] 
MONGO_DIR=os.environ['MONGO_DIR'] 
LOGDIR=os.environ['LOGDIR'] 

# Init Logger 
logging.config.fileConfig(os.path.join(KULOKO_DIR,'ini/logconfig.ini'),defaults={'logfilename': os.path.join(LOGDIR,'logging.log')})
logger = logging.getLogger("KULOKO")

# Init mongo
mongo_ini = configparser.ConfigParser()
mongo_ini.read(os.path.join(MONGO_DIR,'ini/mongo_config.ini'), encoding='utf-8')
sys.path.append(os.environ['COMMON_DIR'] )
from mongodb.src.mongo_handler import MongoHandler

# Init API
import  handler.socket_handler as skt_api
private_api_ini = configparser.ConfigParser()
private_api_ini.read(os.path.join(KULOKO_DIR,'ini/private_api.ini'), encoding='utf-8')

general_config_ini = configparser.ConfigParser()
general_config_ini.read(os.path.join(KULOKO_DIR,'ini/config.ini'), encoding='utf-8')



In [38]:
orderbooks = skt_api.Orderbooks(logger, general_config_ini, private_api_ini)
orderbooks.connect(orderbooks.get_public_socket_url(),'BTC')
orderbooks.subscribe()
mongo_db = MongoHandler(mongo_ini['LOCAL'],"orderbook")

--- request header ---
GET /ws/public/v1 HTTP/1.1
Upgrade: websocket
Host: api.coin.z.com
Origin: http://api.coin.z.com
Sec-WebSocket-Key: 2COUA5fIpznGdWnSWh/MpA==
Sec-WebSocket-Version: 13
Connection: upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Sun, 10 Oct 2021 07:50:21 GMT
Connection: upgrade
Upgrade: websocket
Sec-WebSocket-Accept: 2jBJBOLQhEEe4BpbjPDfNcliEus=
X-Cache: Miss from cloudfront
Via: 1.1 fad8a8c1577be9a62b6296607f594964.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: NRT57-P3
X-Amz-Cf-Id: Jt172eIJFHJLl8riOWEtTBVfUcMjEFKw2ASAuY4nQWI5EfCye2owdg==
-----------------------
send: b'\x81\xc2\x83\x19\x93\x16\xf8;\xf0y\xeet\xf2x\xe7;\xa96\xa1j\xe6t\xf0z\xe1\x7f\xe1|\xb1:\xa3;\xf0~\xe2w\xfds\xef;\xa96\xa1v\xe1r\xe6k\xf1y\xecr\xe04\xaf9\xb1e\xfat\xf1y\xef;\xa96\xa1[\xc7U\xa1d'
send: b'\x8a\x80\xd6\xd5n\x8d'


In [39]:
import time
retry_cnt=0
max_retry_cnt=60
print("START")
while True:
    try:
        time.sleep(10)
        data = orderbooks.get()
        print("Dequeued Data:{0}, Retry_cnt={1}".format(len(data),retry_cnt))
        if len(data)==0:
            retry_cnt += 1
            continue
        insert_json =[ orderbooks.convert_shape(_data,5,"json") for _data in data ]
        mongo_db.insert_many(insert_json)
        retry_cnt = 0
    except Exception as e:
        retry_cnt += 1
        if retry_cnt < max_retry_cnt:
            continue
        print("END",e)
        break


START
Data:100, Retry_cnt=0
Data:21, Retry_cnt=0
Data:20, Retry_cnt=0
Data:20, Retry_cnt=0


send: b'\x8a\x80\x85\x0b\xc8/'


Data:20, Retry_cnt=0
Data:19, Retry_cnt=0
Data:19, Retry_cnt=0
Data:21, Retry_cnt=0
Data:20, Retry_cnt=0
Data:22, Retry_cnt=0


send: b'\x8a\x80n\x92|\xda'


Data:22, Retry_cnt=0
Data:20, Retry_cnt=0


KeyboardInterrupt: 

In [40]:
orderbooks.disconnect()

send: b'\x88\x82\xaeOx\x01\xad\xa7'
send: b'\x81\xc4\xeb\xcc\x06f\x90\xeee\t\x86\xa1g\x08\x8f\xee<F\xc9\xb9h\x15\x9e\xaeu\x05\x99\xa5d\x03\xc9\xe0&D\x88\xa4g\x08\x85\xa9jD\xd1\xec$\t\x99\xa8c\x14\x89\xa3i\r\x98\xee*F\xc9\xbf\x7f\x0b\x89\xa3jD\xd1\xec$$\xbf\x8f$\x1b'
error from callback <bound method Socket.on_close of <handler.socket_handler.Orderbooks object at 0x126d31550>>: socket is already closed.
  File "/Users/macico/.pyenv/versions/3.7.6/envs/py37b/lib/python3.7/site-packages/websocket/_app.py", line 344, in _callback
    callback(*args)
  File "../handler/socket_handler.py", line 135, in on_close
    self.ws.send(json.dumps(message))
  File "/Users/macico/.pyenv/versions/3.7.6/envs/py37b/lib/python3.7/site-packages/websocket/_app.py", line 154, in send
    if not self.sock or self.sock.send(data, opcode) == 0:
  File "/Users/macico/.pyenv/versions/3.7.6/envs/py37b/lib/python3.7/site-packages/websocket/_core.py", line 253, in send
    return self.send_frame(frame)
  File "/User

In [18]:
orderbooks.disconnect()

In [41]:
data = orderbooks.get()

In [42]:
data

[{'channel': 'orderbooks',
  'asks': [{'price': '6236580', 'size': '0.0001'},
   {'price': '6238070', 'size': '0.001'},
   {'price': '6239500', 'size': '0.1'},
   {'price': '6239520', 'size': '0.0125'},
   {'price': '6239650', 'size': '0.4'},
   {'price': '6240000', 'size': '0.011'},
   {'price': '6240250', 'size': '2'},
   {'price': '6241120', 'size': '0.0125'},
   {'price': '6241199', 'size': '0.0122'},
   {'price': '6243350', 'size': '2.5'},
   {'price': '6243550', 'size': '0.1'},
   {'price': '6243900', 'size': '0.4'},
   {'price': '6244550', 'size': '2'},
   {'price': '6245000', 'size': '0.101'},
   {'price': '6245350', 'size': '0.0899'},
   {'price': '6245500', 'size': '0.005'},
   {'price': '6245650', 'size': '1'},
   {'price': '6246160', 'size': '0.005'},
   {'price': '6246800', 'size': '2.5'},
   {'price': '6246900', 'size': '0.0776'},
   {'price': '6247000', 'size': '0.0684'},
   {'price': '6247050', 'size': '0.12'},
   {'price': '6247100', 'size': '0.1795'},
   {'price': '62